## Introduction

This is an example to generate quantum mechanically (QM) calculated electrostatics potentials (ESPs) with `factorpol` package.

Calculated QM ESPs will be stored in a SQL-based database for partial charge or atom-centered polarizability fitting.

## Dependencies
- psi4
- ray
- openff-toolkit
- openff-recharge*
- sqlalchemy
- openeye-toolkits

In order to generate polarized QM ESPs, i.e. with imposed electric field, a modified version of `openff-recharge` is used:

```shell
git clone git@github.com:wwilla7/openff-recharge.git
git checkout add-pol-support
cd openff-recharge
pip install -e .
```

In [1]:
import os
cwd = os.getcwd()

import numpy as np

# QWorker is used for set up and carry out QM calculations
from factorpol.qm_worker import QWorker
# StorageHandler is used for query and store calculated QM properties
from factorpol.utilities import flatten_a_list, StorageHandler
# use openff-toolkit to process dataset
from openff.toolkit import Molecule, ForceField

## Prepare dataset

We use methanol and ethene as dataset for all examples. Resulted parameters, such as polarizabilities, partial charges, and BCC-dPol parameters are only meant for **demonstration**. 

In [2]:
smiles = ["CO", "C=C"]
dataset = [Molecule.from_smiles(s) for s in smiles]
_ = [offmol.generate_conformers(n_conformers=1) for offmol in dataset]

Take a look at the dataset

In [3]:
dataset[0]

NGLWidget()

In [4]:
dataset[1]

NGLWidget()

## Prepare QM calculations

For this QM actor, we will deploy two `ray` workers and provide four cores for each worker to use.

In [5]:
qworker = QWorker(n_worker=2, thread_per_worker=4)

2023-03-30 16:42:44,264	INFO worker.py:1553 -- Started a local Ray instance.


### Baseline QM ESPs
We start with baseline QM ESPs, which doesn't not have imposed external electric field. <br>

> QM level of theory <br>
> MP2/aug-cc-pvtz <br>
> MSK grid setting:  1/4 

In [6]:
ret = qworker.start(
    dataset=dataset,
    method="mp2",
    basis="aug-cc-pvtz",
    wd=os.path.join(cwd, "data_qm"),
    n_conf=1,
    msk_density=1.0,
    msk_layers=4.0,
    external_field=np.zeros(3),
)

[0. 0. 0.]


(_worker pid=2538633) /home/wwilla/mambaforge/envs/factorpol/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.2
(_worker pid=2538633)   warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
(_worker pid=2538633) /home/wwilla/mambaforge/envs/factorpol/lib/python3.9/site-packages/numpy/core/getlimits.py:518: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
(_worker pid=2538633)   setattr(self, word, getattr(machar, word).flat[0])
(_worker pid=2538633) /home/wwilla/mambaforge/envs/factorpol/lib/python3.9/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
(_worker pid=2538633)   return self._float_to_str(self.smallest_subnormal)
(_worker pid=2538633) /home/wwilla/mambaforge/envs/factorpol/lib/python3.9/site-packages/numpy/core/getlimits.py

In [7]:
print(f"Number of MoleculeESPRecords generated:\t {len(ret)}")

Number of MoleculeESPRecords generated:	 2


### Polarized QM ESPs with imposed electric field
We put an imposed external electric field on top of molecules to generate reference polarized QM ESPs for deriving polarizability. <br>

In this example, we impose electric field on the direction of `x+`, with a magnitude of 0.01 a.u. 

> QM level of theory <br>
> MP2/aug-cc-pvtz <br>
> MSK grid setting:  1/4 
> External electric field: [0.01, 0.0, 0.0]

In production, we generate 6 sets of polarized QM ESPs to derive polarizabilities.

In [8]:
external_efield = np.array([0.01, 0.0, 0.0])
ret_polarized = qworker.start(
    dataset=dataset,
    method="mp2",
    basis="aug-cc-pvtz",
    wd=os.path.join(cwd, "data_qm"),
    n_conf=1,
    msk_density=1.0,
    msk_layers=4.0,
    external_field=external_efield,
)

[0.01 0.   0.  ]


In [9]:
print(f"Number of MoleculeESPRecords generated:\t {len(ret_polarized)}")

Number of MoleculeESPRecords generated:	 2


In [10]:
total_records = flatten_a_list(flatten_a_list(qworker.records))
print(f"Total of MoleculeESPRecords generated:\t {len(total_records)}")

Total of MoleculeESPRecords generated:	 4


## Save QM results

In [11]:
st = StorageHandler(local_path=os.path.join(cwd, "data_tmp"))
ses = st.session("factorpol_examples")

Creating new database at postgresql://localhost:5432/factorpol_examples


In [12]:
storage = qworker.store(my_session=ses, records=total_records, )

In [13]:
print(f"Number of records stored:\t {len(storage)}")

Number of records stored:	 4


### Just in case you want to drop the database and start over

In [15]:
# from sqlalchemy_utils import drop_database
# drop_database(ses.bind.url)